In [12]:
import cupy as cp
import numpy as np
from math import floor
import cudaq
import labs_utils as quantum_utils
import matplotlib.pyplot as plt
import random
from collections import deque
from itertools import product


In [ ]:
def get_interactions_cpu(N):
    """
    Generates the interaction sets G2 and G4 based on the loop limits in Eq. 15.
    Returns standard 0-based indices as lists of lists of ints.
    
    Args:
        N (int): Sequence length.
        
    Returns:
        G2: List of lists containing two body term indices
        G4: List of lists containing four body term indices
    """
    # TODO - complete the loops below to compute G2 and G4 indicies

    # Two-body interactions G2
    G2 = []
    for i in range(N - 2):
        max_k = floor((N - 1 - i) / 2)
        for k in range(1, max_k + 1):
            G2.append([i, i + k])
    
    # Four-body interactions G4
    G4 = []
    for i in range(N - 3):
        max_t = floor((N - 2 - i) / 2)
        for t in range(1, max_t + 1):
            max_k = N - 1 - i - t
            for k in range(t + 1, max_k + 1):
                G4.append([i, i + t, i + k, i + k + t])
                
    return G2, G4


# Test the function
print("EXERCISE 4: Testing Interaction Index Generation")
print("=" * 50)

for test_N in [100, 500]:
    G2, G4 = get_interactions_cpu(test_N)
    print(f"\nN = {test_N}:")
    print(f"  G2 (2-body): {len(G2)} interactions")
    print(f"  G4 (4-body): {len(G4)} interactions")
    if test_N <= 7:
        print(f"  G2 = {G2}")
        print(f"  G4 = {G4}")

EXERCISE 4: Testing Interaction Index Generation

N = 100:
  G2 (2-body): 2450 interactions
  G4 (4-body): 79625 interactions


In [ ]:
# =============================================================================
# EXERCISE 3 SOLUTION: CUDA-Q Kernels for 2-Qubit and 4-Qubit Rotations
# =============================================================================

# TODO  Write CUDA-Q kernels to apply the 2 and 4 qubit operators. 

@cudaq.kernel
def two_qubit_ryz(q0: cudaq.qubit, q1: cudaq.qubit, theta: float):
    """R_YZ(θ) = exp(-i θ/2 Y⊗Z)"""
    rx(1.5707963267948966, q0)   # π/2
    h(q1)
    x.ctrl(q0, q1)
    rz(theta, q1)
    x.ctrl(q0, q1)
    h(q1)
    rx(-1.5707963267948966, q0)  # -π/2

@cudaq.kernel  
def two_qubit_rzy(q0: cudaq.qubit, q1: cudaq.qubit, theta: float):
    """R_ZY(θ) = exp(-i θ/2 Z⊗Y)"""
    h(q0)
    rx(1.5707963267948966, q1)
    x.ctrl(q0, q1)
    rz(theta, q1)
    x.ctrl(q0, q1)
    rx(-1.5707963267948966, q1)
    h(q0)

@cudaq.kernel
def four_qubit_ryzzz(q0: cudaq.qubit, q1: cudaq.qubit, q2: cudaq.qubit, q3: cudaq.qubit, theta: float):
    """R_YZZZ(θ) = exp(-i θ/2 Y⊗Z⊗Z⊗Z)"""
    rx(1.5707963267948966, q0)
    h(q1)
    h(q2)
    h(q3)
    x.ctrl(q0, q1)
    x.ctrl(q1, q2)
    x.ctrl(q2, q3)
    rz(theta, q3)
    x.ctrl(q2, q3)
    x.ctrl(q1, q2)
    x.ctrl(q0, q1)
    h(q3)
    h(q2)
    h(q1)
    rx(-1.5707963267948966, q0)

@cudaq.kernel
def four_qubit_zyzz(q0: cudaq.qubit, q1: cudaq.qubit, q2: cudaq.qubit, q3: cudaq.qubit, theta: float):
    """R_ZYZZ(θ) = exp(-i θ/2 Z⊗Y⊗Z⊗Z)"""
    h(q0)
    rx(1.5707963267948966, q1)
    h(q2)
    h(q3)
    x.ctrl(q0, q1)
    x.ctrl(q1, q2)
    x.ctrl(q2, q3)
    rz(theta, q3)
    x.ctrl(q2, q3)
    x.ctrl(q1, q2)
    x.ctrl(q0, q1)
    h(q3)
    h(q2)
    rx(-1.5707963267948966, q1)
    h(q0)

@cudaq.kernel
def four_qubit_zzyz(q0: cudaq.qubit, q1: cudaq.qubit, q2: cudaq.qubit, q3: cudaq.qubit, theta: float):
    """R_ZZYZ(θ) = exp(-i θ/2 Z⊗Z⊗Y⊗Z)"""
    h(q0)
    h(q1)
    rx(1.5707963267948966, q2)
    h(q3)
    x.ctrl(q0, q1)
    x.ctrl(q1, q2)
    x.ctrl(q2, q3)
    rz(theta, q3)
    x.ctrl(q2, q3)
    x.ctrl(q1, q2)
    x.ctrl(q0, q1)
    h(q3)
    rx(-1.5707963267948966, q2)
    h(q1)
    h(q0)

@cudaq.kernel
def four_qubit_zzzy(q0: cudaq.qubit, q1: cudaq.qubit, q2: cudaq.qubit, q3: cudaq.qubit, theta: float):
    """R_ZZZY(θ) = exp(-i θ/2 Z⊗Z⊗Z⊗Y)"""
    h(q0)
    h(q1)
    h(q2)
    rx(1.5707963267948966, q3)
    x.ctrl(q0, q1)
    x.ctrl(q1, q2)
    x.ctrl(q2, q3)
    rz(theta, q3)
    x.ctrl(q2, q3)
    x.ctrl(q1, q2)
    x.ctrl(q0, q1)
    rx(-1.5707963267948966, q3)
    h(q2)
    h(q1)
    h(q0)

print("✓ Exercise 3 Complete: All CUDA-Q kernels defined")

In [ ]:
@cudaq.kernel
def trotterized_circuit(N: int, G2: list[list[int]], G4: list[list[int]], steps: int, dt: float, T: float, thetas: list[float]):
    
    reg = cudaq.qvector(N)
    h(reg)

    # TODO - write the full kernel to apply the trotterized circuit

    for step_idx in range(steps):
        theta = thetas[step_idx]
        
        # Apply 2-body terms
        for pair in G2:
            i0 = pair[0]
            i1 = pair[1]
            two_qubit_ryz(reg[i0], reg[i1], 4.0 * theta)
            two_qubit_rzy(reg[i0], reg[i1], 4.0 * theta)
        
        # Apply 4-body terms
        for quartet in G4:
            i0 = quartet[0]
            i1 = quartet[1]
            i2 = quartet[2]
            i3 = quartet[3]
            four_qubit_ryzzz(reg[i0], reg[i1], reg[i2], reg[i3], 8.0 * theta)
            four_qubit_zyzz(reg[i0], reg[i1], reg[i2], reg[i3], 8.0 * theta)
            four_qubit_zzyz(reg[i0], reg[i1], reg[i2], reg[i3], 8.0 * theta)
            four_qubit_zzzy(reg[i0], reg[i1], reg[i2], reg[i3], 8.0 * theta)




In [ ]:
cudaq.set_target("nvidia")

N = 30
pop_size = 30
T, n_steps = 1.0, 1
dt = T / n_steps

G2, G4 = get_interactions_cpu(N)

# 2. Prepare your batch of theta schedules
# Each schedule is a list of floats (one for each Trotter step)
theta_batch = []
for _ in range(8): # Let's say we want 8 different schedules
    # This creates a list of floats: [theta_t1, theta_t2, ... theta_tn]
    schedule = [quantum_utils.compute_theta(t, dt, T, N, G2, G4) 
                for t in np.linspace(0, T, n_steps)]
    theta_batch.append(schedule)

# Create a list of "Futures" (promises of a result)
futures = []
for schedule in theta_batch:
    # Use .sample_async to send the job to the GPU without waiting
    future = cudaq.sample_async(
        trotterized_circuit,
        N, G2, G4, n_steps, dt, T, schedule,
        shots_count = pop_size * 5
    )
    futures.append(future)

# Now, wait for all of them to finish at once
results = [f.get() for f in futures]


quantum_population = []
for res in results:
    if len(quantum_population) >= pop_size:
        break
    for bitstring, count in res.items():
        if len(quantum_population) >= pop_size:
            break
        # Only add as many as needed to fill the population
        num_to_add = min(count, pop_size - len(quantum_population))
        seq = np.array([1 if b == '0' else -1 for b in bitstring])
        quantum_population.extend([seq] * num_to_add)
